In [5]:
import backtrader as bt
import yfinance as yf
import pandas as pd

# ───── LADDA OCH RENGÖR DATA 100% KOMPATIBELT ─────
ticker = "INVE-B.ST"
df = yf.download(ticker, start="2015-01-01", end="2025-12-31", progress=False)

# Fixar exakt det fel du fick
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]          # bara dessa 5 kolumner
df.columns = ['open', 'high', 'low', 'close', 'volume']      # små bokstäver = Backtrader älskar detta
df.index.name = 'datetime'

# ───── STRATEGI ─────
class GoldenCross(bt.Strategy):
    params = (('short', 20), ('long', 50),)

    def _init_(self):
        self.sma_short = bt.ind.SMA(period=self.p.short)
        self.sma_long  = bt.ind.SMA(period=self.p.long)
        self.crossover = bt.ind.CrossOver(self.sma_short, self.sma_long)

    def next(self):
        if not self.position and self.crossover > 0:
            self.buy(size=self.broker.getcash() // self.data.close[0])
        elif self.position and self.crossover < 0:
            self.close()

# ───── OPTIMERING ─────
cerebro = bt.Cerebro(optreturn=True, stdstats=False)
cerebro.addstrategy(GoldenCross,
                    short=range(10, 51, 5),
                    long=range(50, 201, 15))

cerebro.adddata(bt.feeds.PandasData(dataname=df))
cerebro.broker.setcash(100000)
cerebro.broker.setcommission(commission=0.0005)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')

print("Kör optimering – 80 kombinationer...")
results = cerebro.run()[0]

# ───── BÄSTA RESULTATET ─────
best = max(results, key=lambda x: x.analyzers.sharpe.get_analysis().get('sharperatio', -99))
final = best.broker.getvalue()
sharpe = best.analyzers.sharpe.get_analysis().get('sharperatio', 0)

print(f"\nBÄSTA PARAMETRAR FÖR {ticker}")
print(f"SMA kort: {best.params.short} dagar")
print(f"SMA lång:  {best.params.long} dagar")
print(f"Sharpe-ratio: {sharpe:.2f}")
print(f"100 000 kr → {final:,.0f} kr (+{(final/100000-1)*100:.1f}%)")

C:\Users\Harout\AppData\Local\Temp\ipykernel_9796\3648175523.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2015-01-01", end="2025-12-31", progress=False)


Kör optimering – 80 kombinationer...


AttributeError: 'Lines_LineSeries_LineIterator_DataAccessor_StrategyBase_Strategy_GoldenCross' object has no attribute 'crossover'